# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [ ]:
data = db.read_text('data/reviews_full/reviews_*.json').map(json.loads)
data.take(5)

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [ ]:
data_r = db.read_text('data/reviews_full/reviews_*.json', include_path = True)
data_r.take(3)

In [4]:
def con(pr):
    json_s, rating = pr
    json_d = json.loads(json_s)
    json_d['rating'] = int(re.search(r'(reviews_)\d', pr[1]).group()[-1])
    return json_d


def con_star(json_s, rating):
    json_d = json.loads(json_s)
    json_d['rating'] = int(re.search(r'(reviews_)\d', rating).group()[-1])
    return json_d

In [ ]:
data_n = data_r.starmap(con_star)
data_n.take(3)

3. Посчитайте количество отзывов в исходном датасете.

In [63]:
data_n.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [7]:
def filter_date(obj):
    date = int(obj['date'][:4])
    return date == 2014 or date == 2015

In [ ]:
data_4 = data_n.filter(filter_date)
data_4.take(3)

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [9]:
def string_prep(obj):
    review = obj['review'].lower().strip()
    obj['review'] = ''.join(c for c in review if c.isalpha() or c == ' ')
    return obj

In [10]:
data_5 = data_4.map(string_prep)

In [ ]:
data_5.take(3)

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [88]:
data_5.count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [ ]:
# что выводит промежуточное значение
data_5.groupby(lambda obj: obj['rating']).compute()

In [114]:
# Способ 1 
data_5.groupby(lambda obj: obj['rating']).starmap(lambda rating, arr: (rating, len(arr))).compute()

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

In [126]:
# Способ 2
dict(data_5.pluck('rating').frequencies())

{0: 42472, 1: 9246, 2: 9380, 3: 26532, 4: 119413, 5: 528231}

8. Найдите среднее значение `rating` в выборке

In [120]:
data_5.pluck('rating').mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [128]:
def max_len_elm(a, b):
    return a if len(a['review']) > len(b['review']) else b

In [ ]:
# что выводит промежуточное значение
data_5.foldby(lambda obj: obj['rating'], binop=max_len_elm, combine=max_len_elm).compute()

In [132]:
data_5.foldby(lambda obj: obj['rating'], binop=max_len_elm, combine=max_len_elm).starmap(
                                                lambda rating, obj: (rating, len(obj['review']))).compute()

[(0, 6548), (1, 2868), (2, 2841), (3, 3174), (4, 6548), (5, 5343)]